Import packages

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from prophet import Prophet

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6) # default figure size
plt.rcParams['font.size'] = 12           # default font size
plt.rcParams['font.family'] = 'Times New Roman'

Data preparation

In [ ]:
df = pd.read_csv('../data/energydata_complete_cleaned.csv', parse_dates=['date'], index_col='date')
df.sort_index(inplace=True)
print('Data shape: ', df.shape)

# Define features and target variable
features = ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 
            'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 
            'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 
            'Visibility', 'Tdewpoint', 'hour_of_day', 'day_of_week', 
            'is_weekend', 'hour_sin', 'hour_cos', 'day_of_week_sin', 
            'day_of_week_cos', 'Appliances_lag1', 'Appliances_rolling_mean_6']
target = 'Appliances'
print('Feature number: ', len(features))

# For prophet, we need to rename the columns
prophet_df = df.reset_index().rename(columns={'date': 'ds', target: 'y'})
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

# Split the data into training and testing sets
train_size = int(len(prophet_df) * 0.7)
train_prophet_df = prophet_df.iloc[:train_size].copy()
test_prophet_df = prophet_df.iloc[train_size:].copy()

print('Training data shape for Prophet: ', train_prophet_df.shape)
print('Testing data shape for Prophet: ', test_prophet_df.shape)

Data shape:  (19591, 37)
Feature number:  34
Training data shape:  (13713, 34)
Testing data shape:  (5878, 34)


In [10]:
model_results = {}

Prophet

In [ ]:
prophet_model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=False, # no yearly seasonality due to four months data
    changepoint_prior_scale=0.05
)